<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/Deep-Learning-CNN-Projects/blob/master/FlowerImages-TFLite-Quantization-MobileNet/flower_images_tflite_quantization_mobilenetv2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [26]:
import numpy as np
import os
import pathlib
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [27]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

data_dir = tf.keras.utils.get_file(
    'flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir) / "flower_photos"

# Create Datasets

In [28]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

# Train ds
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, validation_split=0.2, subset="training", seed=123,
    image_size=IMAGE_SIZE, batch_size=BATCH_SIZE
)

# Get class names before applying transformations
class_names = train_ds.class_names

# Create validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, validation_split=0.2, subset="validation", seed=123,
    image_size=IMAGE_SIZE, batch_size=BATCH_SIZE
)

# Preprocess images based on MobilNetV2
def preprocess(x, y):
    return tf.keras.applications.mobilenet_v2.preprocess_input(x), y

# Optimize datasets
train_ds = train_ds.map(preprocess).cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).cache().prefetch(tf.data.AUTOTUNE)

print(f"Classes: {class_names}")

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.
Classes: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


# Load a Pre-Trained Model

In [29]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# Freeze base layers
base_model.trainable = False

# Add new heads
def add_heads(base_model, image_size):
    inputs = tf.keras.Input(shape=image_size)
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(len(class_names), activation="softmax")(x)
    return tf.keras.Model(inputs, outputs)

# Compile
model = add_heads(base_model, IMAGE_SIZE + (3,))
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [30]:
base_model.summary()

Model: "mobilenetv2_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [31]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,264,389 (8.64 MB)

 Trainable params: 6,405 (25.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [32]:
# Training
history = model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds
)

# Save model
KERAS_MODEL_PATH = "flower_model.keras"
model.save(KERAS_MODEL_PATH)
print(f"Model Saved to {KERAS_MODEL_PATH}")

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 34s 191ms/step - accuracy: 0.5480 - loss: 1.1483 - val_accuracy: 0.8610 - val_loss: 0.4478
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.8575 - loss: 0.4150 - val_accuracy: 0.8747 - val_loss: 0.3606
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.8840 - loss: 0.3449 - val_accuracy: 0.8733 - val_loss: 0.3572
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.9102 - loss: 0.2817 - val_accuracy: 0.8937 - val_loss: 0.3084
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.9208 - loss: 0.2479 - val_accuracy: 0.8951 - val_loss: 0.3021
Model Saved to flower_model.keras


# Quantization

In [33]:
# Convert keras model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Activate optimizer
# this optimizer converts float32 to int8
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert model
tflite_quant_model = converter.convert()

# Save quantized model
QUANT_MODEL_PATH = "flower_model_quant.tflite"
with open(QUANT_MODEL_PATH, 'wb') as f:
    f.write(tflite_quant_model)
print(f"Quantized model saved to {QUANT_MODEL_PATH}")

Saved artifact at '/tmp/tmpevay6dam'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_472')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  137423007792208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007789328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007789136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007792016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007788560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007791632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007788752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007788176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007786832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137423007789712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1374230077

# Evaluation (Benchmark)

In [34]:
# compute model's volume
keras_size = os.path.getsize(KERAS_MODEL_PATH) / (1024 * 1024)
quant_size = os.path.getsize(QUANT_MODEL_PATH) / (1024 * 1024)

# Calculate main model's accuracy
_, keras_accuracy = model.evaluate(val_ds)

# Calculate quantized model accuracy
print("\nEvaluating TFLite Model.")

def evaluate_tflite(model_path, dataset):
    # Initialize TFLite interpreter
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_index = input_details[0]['index']
    output_index = output_details[0]['index']

    correct = 0
    total = 0

    # Test on the entire Validation data (image by image)
    for images, labels in dataset.unbatch():
        # Prepare the image (add Batch dimension)
        # TFLite expects input (1, 224, 224, 3)
        img = tf.expand_dims(images, axis=0).numpy()
        label = labels.numpy()

        # Inject the image into the model
        interpreter.set_tensor(input_index, img)
        # Execute
        interpreter.invoke()
        # Get the output
        output = interpreter.get_tensor(output_index)

        prediction = np.argmax(output[0])

        if prediction == label:
            correct += 1
        total += 1

    return correct / total

tflite_acc = evaluate_tflite(QUANT_MODEL_PATH, val_ds)
print("Evaluation Finished...")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8912 - loss: 0.3173

Evaluating TFLite Model.
Evaluation Finished...


In [35]:
# Calculate the difference (new model minus old model)
diff = tflite_acc - keras_accuracy

# Determine the report text based on whether the difference is positive or negative
if diff >= 0:
    change_label = "Accuracy Gain"
    change_value = f"+{diff*100:.2f}%"
else:
    change_label = "Accuracy Drop"
    change_value = f"{abs(diff)*100:.2f}%"

# Print updated report
print("\n" + "="*50)
print("             MODEL Quantization REPORT")
print("="*50)
print(f"{'Metric':<20} | {'Original (Keras)':<15} | {'Quantized (TFLite)':<15}")
print("-" * 56)
print(f"{'Size':<20} | {keras_size:.2f} MB        | {quant_size:.2f} MB")
print(f"{'Accuracy':<20} | {keras_accuracy:.2%}         | {tflite_acc:.2%}")
print("-" * 56)
print(f"Result: The model is {keras_size / quant_size:.1f}x smaller!")
print(f"{change_label}: {change_value}")
print("="*50)


             MODEL Quantization REPORT
Metric               | Original (Keras) | Quantized (TFLite)
--------------------------------------------------------
Size                 | 9.25 MB        | 2.40 MB
Accuracy             | 89.51%         | 89.37%
--------------------------------------------------------
Result: The model is 3.9x smaller!
Accuracy Drop: 0.14%
